In [2]:
import requests
import networkx as nx
import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objs as go

%matplotlib inline

In [19]:
id = 63399107
access_token = 'ba750c0fba750c0fba750c0ff2ba1f5397bba75ba750c0fe6b885ae3f36b2fe81010b1d'

In [20]:
res_users = requests.get('https://api.vk.com/method/users.get?user_ids={0}&access_token={1}&v=5.95'.format(id, access_token))
my_id = res_users.json()['response'][0]['id']

In [22]:
def ids_friends(user_ids):
    friends_url = 'https://api.vk.com/method/friends.get?user_id={}&access_token={}&v=5.95'
    json_response = requests.get(friends_url.format(user_ids, access_token)).json()
    if json_response.get('Error'):
        print(json_response.get('Error'))
        return list()
    return json_response

In [23]:
friend_ids = ids_friends(id)
friend_ids['response']['items'][0]

friends_of_friend = ids_friends(friend_ids['response']['items'][0])
friends_of_friend['response']['items'][0]

239993

In [25]:
graph = {}

friends = ids_friends(my_id)['response']['items']
graph[my_id] = friends

for f in friends:
    friends_of_friend = ids_friends(f)
    if 'response' in friends_of_friend:
        
        graph[f] = friends_of_friend['response']['items']



In [27]:
def about_friends(user_ids):
    friends_url = 'https://api.vk.com/method/users.get?user_id={}&access_token={}&fields=first_name&v=5.95'
    json_response = requests.get(friends_url.format(user_ids, access_token)).json()
    if json_response.get('Error'):
        print(json_response.get('Error'))
        return list()
    return json_response

In [28]:
def name_surname(id):
    info = about_friends(id)['response'][0]
    return info['first_name'] + ' ' + info['last_name']

In [29]:
team = [63399107, 32703581, 86991442]

In [30]:
dict_names = {}
for friend in team:
    dict_names[friend] = name_surname(friend)
    for item in graph[friend]:
        dict_names[item] = name_surname(item)

In [31]:
graph_names = {}

for friend in team:
    graph_names[dict_names[friend]] = []
    for item in graph[friend]:
        graph_names[dict_names[friend]].append(dict_names[item])

In [ ]:
from py2neo import Graph, Node, Relationship
g = Graph("bolt://localhost:7687")
for friend in graph_names:
    a = Node("Person", name=friend)
    for item in graph_names[friend]:
        b = Node("Person", name=item)
        KNOWS = Relationship.type("IS_FRIEND")
        g.merge(KNOWS(a, b), "Person", "name")